In [5]:
import re
import sys
import pandas as pd
from datetime import datetime


In [6]:
pd.options.display.max_columns= None
pd.options.display.max_colwidth= None
pd.options.display.max_rows = None

In [7]:
# ngram_analysis.py

from sklearn.feature_extraction.text import CountVectorizer
# from hazm import Normalizer, word_tokenize, Stemmer, Lemmatizer, stopwords_list

from connect_to_database_func import connect_db

# ---------------------------
# 1) Fetch comments from DB
# ---------------------------
def fetch_comments(sentiment=None, min_len=2, limit=None):
    """
    Fetch comments (id, title, grade, description, sentiment_result) from DB.
    Optionally filter by sentiment ('negative', 'positive', etc.) and non-empty text.
    """
    conn = connect_db()
    cur = conn.cursor()

    base = """
        SELECT id, title, grade, description, COALESCE(sentiment_result, '') as sentiment_result
        FROM comments
        WHERE description IS NOT NULL
          AND trim(description) <> ''
    """
    args = []
    if sentiment:
        base += " AND lower(sentiment_result) = lower(%s)"
        args.append(sentiment)
    base += " ORDER BY id ASC"
    if limit:
        base += " LIMIT %s"
        args.append(limit)

    cur.execute(base, tuple(args))
    rows = cur.fetchall()
    cur.close()
    conn.close()

    df = pd.DataFrame(rows, columns=["id", "title", "grade", "description", "sentiment_result"])
    # drop very short strings
    df = df[df["description"].str.len() >= min_len].reset_index(drop=True)
    return df






In [3]:
df = fetch_comments()

In [4]:
df

,id,title,grade,description,sentiment_result
0,1,پرداخت قبض,3,جایی نداره که من بنویسم این موبایل به نام چه ک...,no sentiment expressed
1,2,پرداخت قبض,3,به من نگفت که شماره تلفن را باید با کد شهر وار...,negative
2,3,پرداخت قبض,3,قبض تلفن ثابت رو پرداخت کردم. به من نگفت با پی...,negative
3,4,پرداخت قبض,4,پس از پرداخت قبض و در حین باز شدن منوی تجربه، ...,negative
4,5,سایر,1,منوی حالات نمایش بصورت دوحالت تاریک و روشن میب...,negative
...,...,...,...,...,...
6640,14918,انتقال وجه,5,خیلی عالی بود\n ازکار کردن بااپ لذت بردم,very positive
6641,14927,خرید شارژ,5,عالی بود وباسرعت زیاد,very positive
6642,14937,انتقال وجه,4,رسید پیچیده است میتواند مفهومی تر مینمال‌تر با...,no sentiment expressed
6643,14940,انتقال وجه,5,عالی بود,very positive


In [ ]:
# ------------------------------------
# 2) Persian normalization/tokenizing
# ------------------------------------
_normalizer = Normalizer(persian_numbers=True, remove_diacritics=True)
_stemmer = Stemmer()
_lem = Lemmatizer()

# Base stopwords (Hazm) + a few domain ones you might not want in n-grams
BASE_STOPWORDS = set(stopwords_list()) | {
    "بانک", "ملت", "بانکملت", "دیما", "دیم", "اپ", "اپلیکیشن", "برنامه",
    "سلام", "ممنون", "لطفا", "لطفاً", "خواهش", "خیلی", "کردم", "میشه", "میشود",
    # add more after inspecting top n-grams
}

_url_re = re.compile(r"https?://\S+|www\.\S+")
_handle_re = re.compile(r"[@#]\S+")
_num_re = re.compile(r"\d+")
_punct_re = re.compile(r"[^\w\s‌]")  # keep Persian ZWNJ (‌)

def clean_and_tokenize(text: str):
    if not isinstance(text, str):
        return []

    # normalize
    t = _normalizer.normalize(text)

    # remove URLs, handles, excessive numbers/punct
    t = _url_re.sub(" ", t)
    t = _handle_re.sub(" ", t)
    t = _num_re.sub(" ", t)
    t = _punct_re.sub(" ", t)

    # tokenize
    tokens = word_tokenize(t)

    # lower, strip, remove stopwords/short tokens
    cleaned = []
    for tok in tokens:
        tok = tok.strip().lower()
        if len(tok) < 2:
            continue
        if tok in BASE_STOPWORDS:
            continue
        # optional lemmatize or stem (pick one; here lemmatize first)
        tok = _lem.lemmatize(tok)
        if tok in BASE_STOPWORDS or len(tok) < 2:
            continue
        cleaned.append(tok)

    return cleaned





In [ ]:
# ---------------------------------------------------
# 3) Count top n-grams (bigrams/trigrams by default)
# ---------------------------------------------------
def top_ngrams(
    texts,
    ngram_range=(2, 3),
    top_k=30,
    min_df=5,
    max_df=0.5,
    max_features=20000
):
    """
    Build a CountVectorizer over cleaned Persian tokens and return top n-grams with counts.
    """
    # Important: we pass a custom tokenizer and disable token_pattern
    vectorizer = CountVectorizer(
        tokenizer=clean_and_tokenize,
        preprocessor=lambda x: x,
        token_pattern=None,
        ngram_range=ngram_range,
        min_df=min_df,
        max_df=max_df,
        max_features=max_features
    )
    X = vectorizer.fit_transform(texts)
    vocab = vectorizer.get_feature_names_out()
    counts = X.sum(axis=0).A1

    df_counts = pd.DataFrame({"ngram": vocab, "count": counts})
    # add n (length of ngram) for clarity
    df_counts["n"] = df_counts["ngram"].str.count(" ") + 1
    df_counts = df_counts.sort_values("count", ascending=False).head(top_k).reset_index(drop=True)
    return df_counts



In [ ]:
# -----------------------------------------
# 4) Save results to DB (optional but handy)
# -----------------------------------------
def save_ngram_stats(df_counts, slice_name="all"):
    """
    Save n-gram stats into a table for dashboarding.
    CREATE TABLE IF NOT EXISTS ngram_stats (
        id SERIAL PRIMARY KEY,
        slice_name TEXT,       -- e.g. 'all', 'negative', 'positive', 'payments'
        n INT,
        ngram TEXT,
        count INT,
        computed_at TIMESTAMP
    );
    """
    if df_counts.empty:
        return

    conn = connect_db()
    cur = conn.cursor()

    # make sure the table exists (idempotent)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS ngram_stats (
            id SERIAL PRIMARY KEY,
            slice_name TEXT,
            n INT,
            ngram TEXT,
            count INT,
            computed_at TIMESTAMP
        );
    """)

    now = datetime.utcnow()
    rows = [
        (slice_name, int(row.n), str(row.ngram), int(row.count), now)
        for _, row in df_counts.iterrows()
    ]

    cur.executemany("""
        INSERT INTO ngram_stats (slice_name, n, ngram, count, computed_at)
        VALUES (%s, %s, %s, %s, %s)
    """, rows)

    conn.commit()
    cur.close()
    conn.close()


In [ ]:
# ------------------------
# 5) Example runner (CLI)
# ------------------------
def main():
    # A) ALL comments
    df_all = fetch_comments()
    df_top_all = top_ngrams(df_all["description"].tolist(), ngram_range=(2,3), top_k=30, min_df=5)
    print("\nTop n-grams (ALL):\n", df_top_all.head(20))
    save_ngram_stats(df_top_all, slice_name="all")

    # B) NEGATIVE comments only (pain points)
    df_neg = fetch_comments(sentiment="negative")
    if not df_neg.empty:
        df_top_neg = top_ngrams(df_neg["description"].tolist(), ngram_range=(2,3), top_k=30, min_df=3)
        print("\nTop n-grams (NEGATIVE):\n", df_top_neg.head(20))
        save_ngram_stats(df_top_neg, slice_name="negative")

    # C) POSITIVE (what users love)
    df_pos = fetch_comments(sentiment="positive")
    if not df_pos.empty:
        df_top_pos = top_ngrams(df_pos["description"].tolist(), ngram_range=(2,3), top_k=30, min_df=3)
        print("\nTop n-grams (POSITIVE):\n", df_top_pos.head(20))
        save_ngram_stats(df_top_pos, slice_name="positive")


if __name__ == "__main__":
    sys.exit(main())
